# Character Animations
This is a bonus. Let's try to animate these characters.

In [2]:
import settings
from model import Story
 
# The last time json data was saved was step 4
story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

In [ ]:
%pip install --upgrade transformers accelerate diffusers imageio-ffmpeg tbb torchvision torch

# Generate list of scene images

Let's stick the characters into a scene so that we can have lots of context.

In [4]:
from PIL import Image
import settings

def place_character_in_scene(image: Image, background: Image=None):
    width = settings.CHARACTER_ANIMATION_WIDTH
    height = settings.CHARACTER_ANIMATION_HEIGHT

    # Create a transparent background
    resize_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    if background:
        # Add background to the image
        scene = background.convert("RGBA")

        # Scale the scene to fit the normal dimensions while preserving aspect ratio
        scene_ratio = max(width / scene.width, height / scene.height)
        new_scene_size = (int(scene.width * scene_ratio), int(scene.height * scene_ratio))
        scene = scene.resize(new_scene_size)
        
        # Calculate the position to paste the scene image onto the background
        scene_offset = ((width - new_scene_size[0]) // 2, (height - new_scene_size[1]))
        
        # Paste the scene image onto the background
        resize_image.paste(scene, scene_offset, scene)


    # Calculate the position to paste the source image onto the background
    src_width, src_height = image.size
    offset = ((width - src_width) // 2, (height - src_height))
    # offset = ((width - src_width) // 2, (height - src_height) // 2)

    # Ensure the image has an alpha channel
    image = image.convert("RGBA")
    
    # Paste the source image onto the background
    resize_image.paste(image, offset, image)

    # Convert the image to RGB
    resize_image = resize_image.convert("RGB")

    return resize_image

In [ ]:
from IPython.display import display, Markdown
import rembg
import os
import random
from utils import deindent

for character in story.characters:
    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}_scene.png"

    # Skip it if we already made it
    if not os.path.exists(dst_image_path):
        act = random.choice(story.acts)
        scene = random.choice(act.scenes)

        src_image_path = f"{settings.STORY_DIR}/step_5/characters/{character.nickname}.png"
        src_image = Image.open(src_image_path)

        # Make the character a bit smaller
        src_image = src_image.resize((src_image.size[0]//3*2, src_image.size[1]//3*2))

        # Remove the background
        src_image = src_image.convert("RGBA")
        src_image = rembg.remove(src_image)

        background = Image.open(f"stories/my_story/step_6/scenes/{scene.scene_id}.png")

        scene_image = place_character_in_scene(src_image, background=background)

        dir_name = os.path.dirname(dst_image_path)
        os.makedirs(dir_name, exist_ok=True)

        scene_image.save(dst_image_path)

    display(Markdown(deindent(f"""
        ---
        ## {character.name}

        ![Animation of {character.name}]({dst_image_path})

    """)))

# Animate Each Character


In [6]:
from model import Story
import settings

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

### Image/Text-to-video

#### CogVideoX-5b-I2V
https://huggingface.co/THUDM/CogVideoX-5b-I2V

In [7]:
from IPython.display import display, Markdown
from model_image2video import image_to_video
from utils import deindent
from PIL import Image

for c in story.characters:
    src_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}_scene.png"
    src_image = Image.open(src_image_path)
    
    prompt = f"slow camera full shot arc around focusing on a {c.gender} {c.race} {c.role} age {c.age}. Hair flowing naturally. {c.physical_appearance}"
    # prompt = f"{c.animation_description}. {scene.background_animation}"
    # prompt = f"An slow epic arc shot to the right while zooming in. Grenades are exploding all around the ninja attacks and dust is flying everywhere. {scene.background_animation}"
    # prompt = f"a {c.gender} {c.race} speaks animatedly, gesturing with their hands. Lasers shoot out of their eyes and an explosion ensues."

    formatted_prompt = "\n".join([f"> {line}" for line in prompt.split("\n")])
    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.gif"
    dst_image_small_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}_small.gif"
    dst_video_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.mp4"
    guidance_scale = 7.0
    
    # Don't re-render the video if it already exists
    # if not os.path.exists(dst_image_small_path):
    image_to_video(prompt=prompt, image=src_image, 
                    video_filename=dst_video_path, 
                    gif_filename=dst_image_path,
                    gif_small_filename=dst_image_small_path,
                    display_video=False, loop_reverse=True, sequences=1)

    display(Markdown(deindent(f"""
        ---
                
        ## {c.name}

        **Role**: {c.role}
                            
        **Physical Apperaance**: {c.physical_appearance}

        **Description**: {c.description}

        **Personality**: {c.personality}

        ### Prompt:

        **guidance_scale**: {guidance_scale}

        {formatted_prompt}

        ![Animation of {c.name}]({dst_image_path})

    """)))

    display(Markdown(f"Full size GIF: [{dst_image_path}](./{dst_image_path})"))
    display(Markdown(f"Full size MP4: [{dst_video_path}](./{dst_video_path})"))

Using model: THUDM/CogVideoX-5b-I2V on device: cuda


2024-11-11 19:58:09.742371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 19:58:09.750004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731373089.758917  295385 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731373089.761549  295385 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 19:58:09.770765: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

...

Full size MP4: [stories/my_story/step_7/characters/earth_rebel.mp4](./stories/my_story/step_7/characters/earth_rebel.mp4)

  0%|          | 0/50 [00:00<?, ?it/s]

---

## Regina 'The Shark' Thornton

**Role**: Antagonist/Corporate Executive

**Physical Apperaance**: Polished, with long, curly blonde hair, piercing blue eyes, and impeccable, high-end business attire.

**Description**: A ruthless, cunning executive exploiting Eon for corporate gain, serving as a primary antagonist on Earth.

**Personality**: Ambitious, manipulative, and devoid of empathy, with a singular focus on profit.

### Prompt:

**guidance_scale**: 7.0

> slow camera full shot arc around focusing on a Female Caucasian Antagonist/Corporate Executive age 35. Hair flowing naturally. Polished, with long, curly blonde hair, piercing blue eyes, and impeccable, high-end business attire.

![Animation of Regina 'The Shark' Thornton](stories/my_story/step_7/characters/corporate_exec.gif)

Full size GIF: [stories/my_story/step_7/characters/corporate_exec.gif](./stories/my_story/step_7/characters/corporate_exec.gif)

Full size MP4: [stories/my_story/step_7/characters/corporate_exec.mp4](./stories/my_story/step_7/characters/corporate_exec.mp4)

# Next Step
~~Onto [Step 8: Prop Descriptions](./8_prop_descriptions.ipynb)~~

Onto [Step 11: Character Voice](./11_character_voice.ipynb)